In [ ]:
#r "nuget:Microsoft.ML, 1.5.2"
#r "nuget:Microsoft.ML.LightGbm, 1.5.2"

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;
using System.Linq;

// Define data structure
public class BinaryClassificationData
{
    [LoadColumn(0)] public float Feature1;
    [LoadColumn(1)] public float Feature2;
    [LoadColumn(2)] public float Feature3;
    [LoadColumn(3)] public float Feature4;
    [LoadColumn(4)] public bool Label;
}

// Generate dummy data
var rnd = new Random();
var data = Enumerable.Range(0, 100)
            .Select(_ => new BinaryClassificationData
            {
                Feature1 = (float)rnd.NextDouble(),
                Feature2 = (float)rnd.NextDouble(),
                Feature3 = (float)rnd.NextDouble(),
                Feature4 = (float)rnd.NextDouble(),
                Label = rnd.Next(2) == 1
            }).ToList();

In [ ]:
using Microsoft.ML.Transforms;

// Create mlContext
var mlContext = new MLContext();

// Create dataView from List
var dataView = mlContext.Data.LoadFromEnumerable(data);

// Define the pipeline
var pipeline = mlContext.Transforms.Concatenate("Features", "Feature1", "Feature2", "Feature3", "Feature4")
    .Append(mlContext.BinaryClassification.Trainers.LightGbm(labelColumnName: "Label", featureColumnName: "Features"));

In [ ]:
// Define the data preprocessing pipeline
var preprocessingPipeline = mlContext.Transforms.Concatenate("Features", "Feature1", "Feature2", "Feature3", "Feature4")
    .Append(mlContext.Transforms.NormalizeMinMax("Features"));

// Define the training pipeline using LightGBM
var trainer = mlContext.BinaryClassification.Trainers.LightGbm();

// Create the training pipeline
var trainingPipeline = preprocessingPipeline.Append(trainer);

// Train the model
var model = trainingPipeline.Fit(dataView);

// Perform predictions on train data to get metrics
var predTransform = model.Transform(dataView);
var metrics = mlContext.BinaryClassification.Evaluate(predTransform);

Console.WriteLine($"Model quality metrics evaluation");
Console.WriteLine($"--------------------------------");
Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");


In [ ]:
mlContext.Model.Save(model, dataView.Schema, "lgbm.mlnet");
Console.WriteLine("Model saved to 'lgbm.mlnet'");

In [ ]:
Console.WriteLine($"Model quality metrics evaluation");
Console.WriteLine($"--------------------------------");
Console.WriteLine($"Accuracy: {metrics.Accuracy:P2}");

In [ ]:
mlContext.Model.Save(model, dataView.Schema, "lgbm.mlnet");
Console.WriteLine("Model saved to 'lgbm.mlnet'");